Library import

In [19]:
import requests   
from bs4 import BeautifulSoup   
import re    

import pandas as pd  
import numpy as np    

from datetime import datetime, timedelta

import warnings    
warnings.filterwarnings('ignore') 

Base variable definition

In [20]:
page_url = "https://snow.myswitzerland.com/snow_reports/piste/?p="   
base_url = "https://snow.myswitzerland.com/"

Initial page scraping

In [10]:
all_data_from_url = []   
list_of_links = []


for page_number in range(1, 8):     
    url = page_url + str(page_number)     
    response = requests.get(url)    
    #print("Status code of the request:", response.status_code)  
    html_content = response.content   
    soup = BeautifulSoup(html_content, 'html.parser')   

    all_resorts_html = soup.find_all('tr', {'class': 'FilterGridTable--row'})  
    #print(f"Number of resorts listed in page {page_number}: ", len(all_resorts_html))
    
    for row in all_resorts_html:  
        all_data_from_url.append(row) 
        resort_url = soup.find_all('a', {'class': 'FilterGridTable--link'}, href=True) 
    
    for link in resort_url:
        href = link['href']
        #print(href)
        full_link = base_url + href   
        
        if full_link not in list_of_links:  
            list_of_links.append(full_link)

len(all_data_from_url)

183

In [11]:
resort_title = []

for row in all_data_from_url:
    headers = row.find_all('div', {"class":"FilterGridTable--title"})    
    data = [header.get_text(strip=True) for header in headers]  
    resort_title.append(data)   
    
resort_df = pd.DataFrame(resort_title, columns = ["Resort"])

In [12]:
resort_df

,Resort
0,Samnaun/Ischgl
1,Engadin & St. Moritz
2,Thyon - 4 Vallées
3,Verbier
4,Klosters
...,...
178,Eischoll
179,Eggiwil
180,Dent-de-Vaulion
181,Torgon-Portes du Soleil


In [13]:
data_locations = [[header.get_text(strip=True) for header in row.find_all('div', {"class":"FilterGridTable--info"})] for row in all_data_from_url]  

resort_df["Location"] = data_locations

In [14]:
resort_df

,Resort,Location
0,Samnaun/Ischgl,[Samnaun – Graubünden]
1,Engadin & St. Moritz,[St. Moritz – Graubünden]
2,Thyon - 4 Vallées,[Les Collons Thyon – Valais]
3,Verbier,[Verbier – Valais]
4,Klosters,[Klosters – Graubünden]
...,...,...
178,Eischoll,[Eischoll – Valais]
179,Eggiwil,[Eggiwil – Bern Region]
180,Dent-de-Vaulion,[Vaulion (Dent-de-Vaulion) – Canton of Vaud]
181,Torgon-Portes du Soleil,[Torgon – Valais]


In [15]:
resort_df["Link"] = list_of_links

In [16]:
resort_df

,Resort,Location,Link
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com//snow_reports/s...
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com//snow_reports/e...
2,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com//snow_reports/t...
3,Verbier,[Verbier – Valais],https://snow.myswitzerland.com//snow_reports/v...
4,Klosters,[Klosters – Graubünden],https://snow.myswitzerland.com//snow_reports/k...
...,...,...,...
178,Eischoll,[Eischoll – Valais],https://snow.myswitzerland.com//snow_reports/e...
179,Eggiwil,[Eggiwil – Bern Region],https://snow.myswitzerland.com//snow_reports/e...
180,Dent-de-Vaulion,[Vaulion (Dent-de-Vaulion) – Canton of Vaud],https://snow.myswitzerland.com//snow_reports/d...
181,Torgon-Portes du Soleil,[Torgon – Valais],https://snow.myswitzerland.com//snow_reports/t...


In [17]:
data_content = []

for row in all_data_from_url:
    content = [content.get_text(strip=True) for content in row.find_all('td', {'class': 'FilterGridTable--cell'})]
    data_content.append(content)

for i, content in enumerate(data_content):
    if len(content) < 3:
        data_content[i] = content + [""] * (3 - len(content))

for i, col in enumerate(["Open Pistes km", "Slope Condition", "Open Lifts"]):
    resort_df[col] = np.array([content[i] for content in data_content])

In [18]:
resort_df

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com//snow_reports/s...,"204.0/239.0 kmPistes, good",goodSlope condition,39/46Lifts open
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com//snow_reports/e...,"121.7/330.3 kmPistes, good",goodSlope condition,26/59Lifts open
2,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com//snow_reports/t...,"85.8/358.0 kmPistes, good",goodSlope condition,19/73Lifts open
3,Verbier,[Verbier – Valais],https://snow.myswitzerland.com//snow_reports/v...,"85.8/358.0 kmPistes, good",goodSlope condition,19/73Lifts open
4,Klosters,[Klosters – Graubünden],https://snow.myswitzerland.com//snow_reports/k...,"84.3/258.9 kmPistes, good",goodSlope condition,19/46Lifts open
...,...,...,...,...,...,...
178,Eischoll,[Eischoll – Valais],https://snow.myswitzerland.com//snow_reports/e...,"0.0/15.0 kmPistes, no info",no infoSlope condition,0/3Lifts open
179,Eggiwil,[Eggiwil – Bern Region],https://snow.myswitzerland.com//snow_reports/e...,"0.0/3.0 kmPistes, no info",no infoSlope condition,0/1Lifts open
180,Dent-de-Vaulion,[Vaulion (Dent-de-Vaulion) – Canton of Vaud],https://snow.myswitzerland.com//snow_reports/d...,"0.0/8.0 kmPistes, no info",no infoSlope condition,0/2Lifts open
181,Torgon-Portes du Soleil,[Torgon – Valais],https://snow.myswitzerland.com//snow_reports/t...,,There are no current reports available,


In [ ]:
scraped_data = []

for link in resort_df["Link"]:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")

    table_rows_resort = soup.find_all("div", {"class": "KeyValueList firstColumnStretchMore"})

    link_data = {}    


    for row in table_rows_resort:
        cont_test = row.find_all("th", {"scope": "row"})
        test_test = row.find_all("td")

        th_values = [content.get_text(strip = True) for content in cont_test]
        td_values = [con.get_text(strip = True) for con in test_test]

        link_data.update(dict(zip(th_values, td_values)))   

    scraped_data.append(link_data)

scraped_df = pd.DataFrame(scraped_data)

resort_df = pd.concat([resort_df, scraped_df], axis=1)

In [ ]:
hours_data = []

for link in resort_df["Link"]:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")

    resort_op_hours = soup.find_all("div", {"class": "ArticleSection", "id": "articlesection-u20"})

    link_data_hours = {}

    for row in resort_op_hours:
        cont_test = row.find_all("th", {"scope": "row"})
        test_test = row.find_all("td")

        th_values = [content.get_text(strip = True) for content in cont_test]
        td_values = [con.get_text(strip = True) for con in test_test]

        link_data_hours.update(dict(zip(th_values, td_values)))

    hours_data.append(link_data_hours)

opening_hours = pd.DataFrame(hours_data)

resort_df = pd.concat([resort_df, opening_hours], axis=1)

In [ ]:
prices_data = []

for link in resort_df["Link"]:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")

    resort_prices = soup.find_all("div", {"class": "ArticleSubSection--content"})

    link_data_prices = {}

    for row in resort_prices:
        cont_test = row.find_all("th", {"scope": "row"})
        test_test = row.find_all("td")

        th_values = [content.get_text(strip=True) for content in cont_test]
        td_values = [con.get_text(strip=True) for con in test_test]
 
        if len(th_values) == 0:    
            continue      
        else:
            break
    
    for key, value in zip(th_values, td_values):
        
        counter = 1
        new_key = key
        while new_key in link_data_prices:
            counter += 1
            new_key = f"{key}_{counter}"

        link_data_prices[new_key] = value

    prices_data.append(link_data_prices)

resort_prices = pd.DataFrame(prices_data)

resort_df = pd.concat([resort_df, resort_prices], axis=1)

In [ ]:
resort_df.to_csv("../Data/resort_stage1.csv", index = False)